# --- Day 16: Packet Decoder ---

In [1]:
from Quiz import *
from copy import deepcopy

### --- Part One ---

As you leave the cave and reach open waters, you receive a transmission from the Elves back on the ship.

The transmission was sent using the Buoyancy Interchange Transmission System (<ins>BITS</ins>), a method of packing numeric expressions into a binary sequence. Your submarine's computer has saved the transmission in hexadecimal https://en.wikipedia.org/wiki/Hexadecimal (your puzzle input).

The first step of decoding the message is to convert the hexadecimal representation into binary. Each character of hexadecimal corresponds to four bits of binary data:

`0 = 0000
1 = 0001
2 = 0010
3 = 0011
4 = 0100
5 = 0101
6 = 0110
7 = 0111
8 = 1000
9 = 1001
A = 1010
B = 1011
C = 1100
D = 1101
E = 1110
F = 1111`

The BITS transmission contains a single __packet__ at its outermost layer which itself contains many other packets. The hexadecimal representation of this packet might encode a few extra `0` bits at the end; these are not part of the transmission and should be ignored.

Every packet begins with a standard header: the first three bits encode the packet __version__, and the next three bits encode the packet __type ID__. These two values are numbers; all numbers encoded in any packet are represented as binary with the most significant bit first. For example, a version encoded as the binary sequence `100` represents the number `4`.

Packets with type ID `4` represent a __literal value__. Literal value packets encode a single binary number. To do this, the binary number is padded with leading zeroes until its length is a multiple of four bits, and then it is broken into groups of four bits. Each group is prefixed by a `1` bit except the last group, which is prefixed by a `0` bit. These groups of five bits immediately follow the packet header. 

For example, the hexadecimal string `D2FE28` becomes:

`110100101111111000101000
VVVTTTAAAAABBBBBCCCCC`

Below each bit is a label indicating its purpose:

- The three bits labeled `V` (`110`) are the packet version, `6`.
- The three bits labeled `T` (`100`) are the packet type ID, `4`, which means the packet is a literal value.
- The five bits labeled `A` (`10111`) start with a `1` (not the last group, keep reading) and contain the first four bits of the number, `0111`.
- The five bits labeled `B` (`11110`) start with a `1` (not the last group, keep reading) and contain four more bits of the number, `1110`.
- The five bits labeled `C` (`00101`) start with a `0` (last group, end of packet) and contain the last four bits of the number, `0101`.
- The three unlabeled `0` bits at the end are extra due to the hexadecimal representation and should be ignored.

So, this packet represents a literal value with binary representation `011111100101`, which is `2021` in decimal.

Every other type of packet (any packet with a type ID other than `4`) represent an __operator__ that performs some calculation on one or more sub-packets contained within. Right now, the specific operations aren't important; focus on parsing the hierarchy of sub-packets.

An operator packet contains one or more packets. To indicate which subsequent binary data represents its sub-packets, an operator packet can use one of two modes indicated by the bit immediately after the packet header; this is called the __length type ID__:

- If the length type ID is `0`, then the next __15__ bits are a number that represents the __total length in bits__ of the sub-packets contained by this packet.
- If the length type ID is `1`, then the next __11__ bits are a number that represents the __number of sub-packets immediately contained__ by this packet.

Finally, after the length type ID bit and the 15-bit or 11-bit field, the sub-packets appear.

For example, here is an operator packet (hexadecimal string `38006F45291200`) with length type ID `0` that contains two sub-packets:

`00111000000000000110111101000101001010010001001000000000
VVVTTTILLLLLLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBBBBBB`

- The three bits labeled `V` (`001`) are the packet version, `1`.
- The three bits labeled `T` (`110`) are the packet type ID, `6`, which means the packet is an operator.
- The bit labeled `I` (`0`) is the length type ID, which indicates that the length is a 15-bit number representing the number of bits in the sub-packets.
- The 15 bits labeled `L` (`000000000011011`) contain the length of the sub-packets in bits, `27`.
- The 11 bits labeled `A` contain the first sub-packet, a literal value representing the number `10`.
- The 16 bits labeled `B` contain the second sub-packet, a literal value representing the number `20`.

After reading 11 and 16 bits of sub-packet data, the total length indicated in `L` (27) is reached, and so parsing of this packet stops.

As another example, here is an operator packet (hexadecimal string `EE00D40C823060`) with length type ID `1` that contains three sub-packets:

`11101110000000001101010000001100100000100011000001100000
VVVTTTILLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBCCCCCCCCCCC`

- The three bits labeled `V` (111) are the packet version, `7`.
- The three bits labeled `T` (011) are the packet type ID, `3`, which means the packet is an operator.
- The bit labeled `I` (`1`) is the length type ID, which indicates that the length is a 11-bit number representing the number of sub-packets.
- The 11 bits labeled `L` (`00000000011`) contain the number of sub-packets, `3`.
- The 11 bits labeled `A` contain the first sub-packet, a literal value representing the number `1`.
- The 11 bits labeled `B` contain the second sub-packet, a literal value representing the number `2`.
- The 11 bits labeled `C` contain the third sub-packet, a literal value representing the number `3`.

After reading 3 complete sub-packets, the number of sub-packets indicated in `L` (`3`) is reached, and so parsing of this packet stops.

For now, parse the hierarchy of the packets throughout the transmission and __add up all of the version numbers__.

Here are a few more examples of hexadecimal-encoded transmissions:

- `8A004A801A8002F478` represents an operator packet (version 4) which contains an operator packet (version 1) which contains an operator packet (version 5) which contains a literal value (version 6); this packet has a version sum of __`16`__.
- `620080001611562C8802118E34` represents an operator packet (version 3) which contains two sub-packets; each sub-packet is an operator packet that contains two literal values. This packet has a version sum of __`12`__.
- `C0015000016115A2E0802F182340` has the same structure as the previous example, but the outermost packet uses a different length type ID. This packet has a version sum of __`23`__.
- `A0016C880162017C3686B18A3D4780` is an operator packet that contains an operator packet that contains an operator packet that contains five literal values; it has a version sum of __`31`__.

Decode the structure of your hexadecimal-encoded BITS transmission; __what do you get if you add up the version numbers in all packets__?

In [2]:
hex_bin = {'0' : "0000" , '1' : "0001" , '2' : "0010" , '3' : "0011" , '4' : "0100" , '5' : "0101" , '6' : "0110" , '7' : "0111" , '8' : "1000" , '9' : "1001" , 'A' : "1010" , 'B' : "1011" , 'C' : "1100" , 'D' : "1101" , 'E' : "1110" , 'F' : "1111"}

In [3]:
def get_n(data , n):

    ret = data[0][:n]
    data[0] = data[0][n:]

    return ret

In [4]:
def transform(inp):
    
    trans = []

    for i in range(len(inp)):

        trans.append(hex_bin.get(inp[i]))

    return "".join(trans)
    

In [5]:
def parse(data):

    global count_versions
    
    version = int(get_n(data , 3) , 2)
    count_versions = count_versions + version
    type_id = int(get_n(data , 3) , 2)

    if (type_id == 4):

        lead = '1'
        num = ""

        while (lead != '0'):

            seg = get_n(data , 5)
            lead = seg[0]
            num = num + "".join(seg[1:])

        num_bin = "0b" + num

        return int(num_bin , 2)

    length_type_id = get_n(data , 1)[0]
    sub_packets = []

    if (length_type_id == '0'):

        subpacketslen = int(get_n(data, 15), 2)
        subpackets = [get_n(data, subpacketslen)]

        while (subpackets[0]):

            sub_packets.append(parse(subpackets))

    else:
        
        for i in range(int(get_n(data, 11), 2)):

            sub_packets.append(parse(data))

#### Test

In [6]:
inp = list(test_input_1)
inp = transform(inp)
count_versions = 0

parse([inp])

print("Version sum of '" , test_input_1 , "' is" , count_versions)

Version sum of ' 8A004A801A8002F478 ' is 16


In [7]:
inp = list(test_input_2)
inp = transform(inp)
count_versions = 0

parse([inp])

print("Version sum of '" , test_input_2 , "' is" , count_versions)

Version sum of ' 620080001611562C8802118E34 ' is 12


In [8]:
inp = list(test_input_3)
inp = transform(inp)
count_versions = 0

parse([inp])

print("Version sum of '" , test_input_3 , "' is" , count_versions)

Version sum of ' C0015000016115A2E0802F182340 ' is 23


In [9]:
inp = list(test_input_4)
inp = transform(inp)
count_versions = 0

parse([inp])

print("Version sum of '" , test_input_4 , "' is" , count_versions)

Version sum of ' A0016C880162017C3686B18A3D4780 ' is 31


#### Answer

In [10]:
inp = list(input_1)
inp = transform(inp)
count_versions = 0

parse([inp])

print("Version sum of '" , input_1 , "' is" , count_versions)

Version sum of ' 00537390040124EB240B3EDD36B68014D4C9ECCCE7BDA54E62522A300525813003560004223BC3F834200CC108710E98031C94C8B4BFFF42398309DDD30EEE00BCE63F03499D665AE57B698F9802F800824DB0CE1CC23100323610069D8010ECD4A5CE5B326098419C319AA2FCC44C0004B79DADB1EB48CE5EB7B2F4A42D9DF0AA74E66468C0139341F005A7BBEA5CA65F3976200D4BC01091A7E155991A7E155B9B4830056C01593829CC1FCD16C5C2011A340129496A7EFB3CA4B53F7D92675A947AB8A016CD631BE15CD5A17CB3CEF236DBAC93C4F4A735385E401804AA86802D291ED19A523DA310006832F07C97F57BC4C9BBD0764EE88800A54D5FB3E60267B8ED1C26AB4AAC0009D8400854138450C4C018855056109803D11E224112004DE4DB616C493005E461BBDC8A80350000432204248EA200F4148FD06C804EE1006618419896200FC1884F0A00010A8B315A129009256009CFE61DBE48A7F30EDF24F31FCE677A9FB018F6005E500163E600508012404A72801A4040688010A00418012002D51009FAA0051801CC01959801AC00F520027A20074EC1CE6400802A9A004A67C3E5EA0D3D5FAD3801118E75C0C00A97663004F0017B9BD8CCA4E2A7030C0179C6799555005E5CEA55BC8025F8352A4B2EC92ADF244128C44014649F52BC01793499EA4CBD4

-------------------------------------

### --- Part Two ---

Now that you have the structure of your transmission decoded, you can calculate the value of the expression it represents.

Literal values (type ID `4`) represent a single number as described above. The remaining type IDs are more interesting:

- Packets with type ID `0` are __sum__ packets - their value is the sum of the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID `1` are __product__ packets - their value is the result of multiplying together the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID `2` are __minimum__ packets - their value is the minimum of the values of their sub-packets.
- Packets with type ID `3` are __maximum__ packets - their value is the maximum of the values of their sub-packets.
- Packets with type ID `5` are __greater than__ packets - their value is __1__ if the value of the first sub-packet is greater than the value of the second sub-packet; otherwise, their value is __0__. These packets always have exactly two sub-packets.
- Packets with type ID `6` are __less than__ packets - their value is __1__ if the value of the first sub-packet is less than the value of the second sub-packet; otherwise, their value is __0__. These packets always have exactly two sub-packets.
- Packets with type ID `7` are __equal to__ packets - their value is __1__ if the value of the first sub-packet is equal to the value of the second sub-packet; otherwise, their value is __0__. These packets always have exactly two sub-packets.

Using these rules, you can now work out the value of the outermost packet in your BITS transmission.

For example:

- `C200B40A82` finds the sum of `1` and `2`, resulting in the value __`3`__.
- `04005AC33890` finds the product of `6` and `9`, resulting in the value __`54`__.
- `880086C3E88112` finds the minimum of `7`, `8`, and `9`, resulting in the value `7`.
- `CE00C43D881120` finds the maximum of `7`, `8`, and `9`, resulting in the value `9`.
- `D8005AC2A8F0` produces `1`, because `5` is less than `15`.
- `F600BC2D8F` produces `0`, because `5` is not greater than `15`.
- `9C005AC2F8F0` produces `0`, because `5` is not equal to `15`.
- `9C0141080250320F1802104A08` produces `1`, because `1` + `3` = `2` * `2`.

__What do you get if you evaluate the expression represented by your hexadecimal-encoded BITS transmission__?

In [11]:
def parse(data):

    version = int(get_n(data , 3) , 2)
    type_id = int(get_n(data , 3) , 2)

    if (type_id == 4):

        lead = '1'
        num = ""

        while (lead != '0'):

            seg = get_n(data , 5)
            lead = seg[0]
            num = num + "".join(seg[1:])

        num_bin = "0b" + num

        return int(num_bin , 2)

    length_type_id = get_n(data , 1)[0]
    sub_packets = []

    if (length_type_id == '0'):

        subpacketslen = int(get_n(data, 15), 2)
        subpackets = [get_n(data, subpacketslen)]

        while (subpackets[0]):

            sub_packets.append(parse(subpackets))

    else:
        
        for i in range(int(get_n(data, 11), 2)):

            sub_packets.append(parse(data))

    if (type_id == 0):

        return sum(sub_packets)

    elif (type_id == 1):

        p = 1

        for x in sub_packets:

            p = p * x

        return p

    elif (type_id == 2):

        return min(sub_packets)

    elif (type_id == 3):

        return max(sub_packets)

    elif (type_id == 5):

        return int(sub_packets[0] > sub_packets[1])

    elif (type_id == 6):

        return int(sub_packets[0] < sub_packets[1])

    elif (type_id == 7):

        return int(sub_packets[0] == sub_packets[1])

#### Test

In [12]:
inp = list(test_input_5)
inp = transform(inp)

res = parse([inp])

print("Result of '" , test_input_5 , "' is" , res)

Result of ' C200B40A82 ' is 3


In [13]:
inp = list(test_input_6)
inp = transform(inp)

res = parse([inp])

print("Result of '" , test_input_6 , "' is" , res)

Result of ' 04005AC33890 ' is 54


In [14]:
inp = list(test_input_7)
inp = transform(inp)

res = parse([inp])

print("Result of '" , test_input_7 , "' is" , res)

Result of ' 880086C3E88112 ' is 7


In [15]:
inp = list(test_input_8)
inp = transform(inp)

res = parse([inp])

print("Result of '" , test_input_8 , "' is" , res)

Result of ' CE00C43D881120 ' is 9


In [16]:
inp = list(test_input_9)
inp = transform(inp)

res = parse([inp])

print("Result of '" , test_input_9 , "' is" , res)

Result of ' D8005AC2A8F0 ' is 1


In [17]:
inp = list(test_input_10)
inp = transform(inp)

res = parse([inp])

print("Result of '" , test_input_10 , "' is" , res)

Result of ' F600BC2D8F ' is 0


In [18]:
inp = list(test_input_1)
inp = transform(inp)

res = parse([inp])

print("Result of '" , test_input_1 , "' is" , res)

Result of ' 8A004A801A8002F478 ' is 15


In [19]:
inp = list(test_input_12)
inp = transform(inp)

res = parse([inp])

print("Result of '" , test_input_12 , "' is" , res)

Result of ' 9C0141080250320F1802104A08 ' is 1


#### Answer

In [20]:
inp = list(input_1)
inp = transform(inp)

res = parse([inp])

print("Result of '" , input_1 , "' is" , res)

Result of ' 00537390040124EB240B3EDD36B68014D4C9ECCCE7BDA54E62522A300525813003560004223BC3F834200CC108710E98031C94C8B4BFFF42398309DDD30EEE00BCE63F03499D665AE57B698F9802F800824DB0CE1CC23100323610069D8010ECD4A5CE5B326098419C319AA2FCC44C0004B79DADB1EB48CE5EB7B2F4A42D9DF0AA74E66468C0139341F005A7BBEA5CA65F3976200D4BC01091A7E155991A7E155B9B4830056C01593829CC1FCD16C5C2011A340129496A7EFB3CA4B53F7D92675A947AB8A016CD631BE15CD5A17CB3CEF236DBAC93C4F4A735385E401804AA86802D291ED19A523DA310006832F07C97F57BC4C9BBD0764EE88800A54D5FB3E60267B8ED1C26AB4AAC0009D8400854138450C4C018855056109803D11E224112004DE4DB616C493005E461BBDC8A80350000432204248EA200F4148FD06C804EE1006618419896200FC1884F0A00010A8B315A129009256009CFE61DBE48A7F30EDF24F31FCE677A9FB018F6005E500163E600508012404A72801A4040688010A00418012002D51009FAA0051801CC01959801AC00F520027A20074EC1CE6400802A9A004A67C3E5EA0D3D5FAD3801118E75C0C00A97663004F0017B9BD8CCA4E2A7030C0179C6799555005E5CEA55BC8025F8352A4B2EC92ADF244128C44014649F52BC01793499EA4CBD402697